# Topik: Analisis Sentimen Opini Publik terhadap Kebijakan Pemindahan Dana Rp 200 Triliun ke Bank Himbara oleh Menteri Keuangan Purbaya melalui Komentar YouTube

## Notebook: Data Scraping dan Filtering

**Anggota Kelompok:**
- 140810230011 - Lukas Austin
- 140810230045 - Devin Suryadi
- 140810230057 - Orlando Bloem Sutono

Mata Kuliah: Data Mining <br>
Kelas: A <br>
Dosen: Bu Helen

---

## 1. Import Library
Import semua library yang diperlukan untuk proses scraping data komentar dari YouTube API.

In [22]:
from dotenv import load_dotenv
import os
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from datetime import datetime
import pandas as pd

print("Library siap digunakan.")

Library siap digunakan.


## 2. Load Youtube API Key
Bagian ini digunakan untuk memuat file `.env` yang berisi API Key YouTube.

In [23]:
load_dotenv()

True

## 3. Inisialisasi API Key
Kode berikut digunakan untuk mengambil nilai API Key dari environment variable dan memverifikasi bahwa kunci berhasil dimuat.


In [24]:
API_KEY = os.getenv("YOUTUBE_API_KEY")

if not API_KEY:
    print("Error: YOUTUBE_API_KEY not found.")
else:
    print("API Key loaded successfully.")

API Key loaded successfully.


## 4. Daftar Video
Bagian ini berisi daftar `video_id` dari video YouTube yang akan digunakan untuk proses scraping komentar.
Setiap video ID merepresentasikan satu video berbeda yang relevan dengan topik.

In [25]:
# The list of video IDs
VIDEO_ID_LIST = [
    '585-A9uh6Qc',
    'lyBu3Ew3ZL4',
    'xaT4GPEQhD0',
    'S-uR-p--OHs',
    'qO3LAcESAIA',
    'HNr2Zkbi_LA',
    '_3J7dktLIm4',
    'tOUgdSNKHkA',
    'qhePd6FzOaw',
    '0wUxI5rLVaw',
    '_qRzTJIDAHA',
    'GQ_YTznY7gU',
    'JpTV4lXHWOo'
]

# Output file
OUTPUT_CSV_FILE = "dataset_raw.csv"

# Scraper settings
COMMENTS_PER_VIDEO = 100
SORT_ORDER = "relevance"

## 5. Inisialisasi YouTube API Client
Dilakukan inisialisasi client API untuk mengakses data publik dari YouTube menggunakan library `googleapiclient.discovery`.

In [26]:
client = build("youtube", "v3", developerKey=API_KEY)

## 6. Scraping Komentar
Ini adalah bagian utama yang melakukan proses pengambilan data komentar dari setiap video.  
Setiap komentar yang diperoleh akan disimpan dalam list, kemudian dikonversi menjadi DataFrame.

In [27]:
all_comments_list = []
total_videos = len(VIDEO_ID_LIST)

if client:
    print(f"Memulai scraping untuk {total_videos} video...")
    
    # Loop untuk setiap video ID
    for i, video_id in enumerate(VIDEO_ID_LIST, 1):
        print(f"[{i}/{total_videos}] Memproses ID: {video_id}...")
        
        try:
            # Meminta 'commentThreads' (komentar top-level)
            request = client.commentThreads().list(
                part="snippet",
                videoId=video_id,
                maxResults=COMMENTS_PER_VIDEO,
                order=SORT_ORDER,
                textFormat="plainText"
            )
            response = request.execute()
            
            count = 0
            # Loop untuk setiap komentar yang didapat
            for item in response.get("items", []):
                # Mengambil data dari 'snippet'
                snippet = item.get('snippet', {}).get('topLevelComment', {}).get('snippet', {})
                
                published_at_raw = snippet.get("publishedAt", "")
                try:
                    published_at = datetime.fromisoformat(published_at_raw.rstrip('Z')).strftime('%Y-%m-%d')
                except ValueError:
                    published_at = published_at_raw # Gunakan teks asli jika format gagal

                # Tambahkan data ke list utama
                all_comments_list.append({
                    "id_comment": item.get('id', ''),
                    "comment": snippet.get('textDisplay', '').replace("\n", " ").replace("\r", " "),
                    "author": snippet.get('authorDisplayName', ''),
                    "like_count": snippet.get('likeCount', 0),
                    "time": published_at,
                    "video_id": video_id
                })
                count += 1

            print(f"> Total: {count} komentar.")
            
        except HttpError as e:
            print(f"> Gagal mengambil data untuk {video_id}. Error: {e}. Lanjut...")

    print("\nDone scraping")

    df_raw = pd.DataFrame(all_comments_list)

else:
    print("Eksekusi dihentikan karena API client gagal dibuat.")

Memulai scraping untuk 13 video...
[1/13] Memproses ID: 585-A9uh6Qc...
> Total: 100 komentar.
[2/13] Memproses ID: lyBu3Ew3ZL4...
> Total: 100 komentar.
[3/13] Memproses ID: xaT4GPEQhD0...
> Total: 100 komentar.
[4/13] Memproses ID: S-uR-p--OHs...
> Total: 100 komentar.
[5/13] Memproses ID: qO3LAcESAIA...
> Total: 100 komentar.
[6/13] Memproses ID: HNr2Zkbi_LA...
> Total: 100 komentar.
[7/13] Memproses ID: _3J7dktLIm4...
> Total: 100 komentar.
[8/13] Memproses ID: tOUgdSNKHkA...
> Total: 100 komentar.
[9/13] Memproses ID: qhePd6FzOaw...
> Total: 100 komentar.
[10/13] Memproses ID: 0wUxI5rLVaw...
> Total: 32 komentar.
[11/13] Memproses ID: _qRzTJIDAHA...
> Total: 7 komentar.
[12/13] Memproses ID: GQ_YTznY7gU...
> Total: 21 komentar.
[13/13] Memproses ID: JpTV4lXHWOo...
> Total: 100 komentar.

Done scraping


In [28]:
df_raw.head()

,id_comment,comment,author,like_count,time,video_id
0,UgwKOUwkackHqtLfbPR4AaABAg,"Seperti langit dan bumi, kerja pak Purbaya dan...",@bettyutami9973,15,2025-10-13,585-A9uh6Qc
1,Ugx_jFD82BwGojRWq4F4AaABAg,"Semoga Purbaya selamat, ngak dikerjain sama ol...",@IndoHog-fe2vd,78,2025-10-12,585-A9uh6Qc
2,UgxeuHpB4tJs3VaxIa94AaABAg,Yg saya heran setiap kali Bang Bennix bilang G...,@misbakulhuda663,88,2025-10-12,585-A9uh6Qc
3,UgzMgVYyDxuC_lAqH3V4AaABAg,nyatanya bank2 himbara masih selektif mengucur...,@muhammadmoechlisin702,20,2025-10-12,585-A9uh6Qc
4,Ugwt2EEv2dZFFy9kkql4AaABAg,Ayooo pak pur ttp semangat bangun indonesia 👍,@Emoxn7568,52,2025-10-12,585-A9uh6Qc


## 7. Menyimpan Hasil Scraping Data
Setelah seluruh komentar berhasil diambil, data disimpan ke file CSV agar dapat digunakan pada tahap preprocessing nantinya.

In [29]:
df_raw.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8')
print(f"Berhasil menyimpan df ke csv")

Berhasil menyimpan df ke csv


## 8. Filtering Data
Digunakan untuk melakukan penyaringan awal menggunakan daftar *keyword* yang relevan dengan topik agar komentar tidak relevan tidak masuk ke data untuk diproses.

In [30]:
# keyword filter
keyword_list = [
    "indonesia", "200", "200t", "menteri", "keuangan", "purbaya", "bank", "usaha",
    "kredit", "rakyat", "ekonomi", "negara", "pemerintah", "korup",
    "umkm", "himbara", "suntik", "suntikan", "triliun", "likuiditas", "dana",
    "harga", "bunga", "uang", "pur", "pak men", "mentri", "menkeu", "pak",
    "awas", "duit", "masyarakat", "bapak", "tepat", "sasaran", "dulu", "gaji", "beli",
    "dirut"
]

print(f"Total keyword untuk filter: {len(keyword_list)}")

Total keyword untuk filter: 39


In [31]:
keyword_pattern = '|'.join(keyword_list)
mask_relevan = df_raw['comment'].str.contains(keyword_pattern, case=False, na=False)

# Data relevan
df_filter = df_raw[mask_relevan]
df_not_used = df_raw[~mask_relevan]

print(f"Data relevan: {len(df_filter)}")
print(f"Data tidak relevan: {len(df_not_used)}")

Data relevan: 924
Data tidak relevan: 136


In [32]:
df_not_used.head()

,id_comment,comment,author,like_count,time,video_id
2,UgxeuHpB4tJs3VaxIa94AaABAg,Yg saya heran setiap kali Bang Bennix bilang G...,@misbakulhuda663,88,2025-10-12,585-A9uh6Qc
7,Ugx5BbOJxUVVP_SK1IR4AaABAg,Bubarkan OJK dan pinjol,@ahmadalisudibyo3601,10,2025-10-12,585-A9uh6Qc
10,UgzMivudK0Yg8PFHU414AaABAg,"Dari 40ribuan subs, now here we are",@RasanyaEnak-eb1jw,12,2025-10-12,585-A9uh6Qc
17,UgxdamfgLoDzVgHh8Rl4AaABAg,Kalau dari video yang udah-udah 5k likes aja p...,@roe_goodknight,20,2025-10-12,585-A9uh6Qc
20,UgzdGewoFJXWotK9Mwp4AaABAg,"Sayang banget, viewer channel ini dipenuhi org...",@oppoajjawowo,9,2025-10-12,585-A9uh6Qc


In [33]:
df_filter.head()

,id_comment,comment,author,like_count,time,video_id
0,UgwKOUwkackHqtLfbPR4AaABAg,"Seperti langit dan bumi, kerja pak Purbaya dan...",@bettyutami9973,15,2025-10-13,585-A9uh6Qc
1,Ugx_jFD82BwGojRWq4F4AaABAg,"Semoga Purbaya selamat, ngak dikerjain sama ol...",@IndoHog-fe2vd,78,2025-10-12,585-A9uh6Qc
3,UgzMgVYyDxuC_lAqH3V4AaABAg,nyatanya bank2 himbara masih selektif mengucur...,@muhammadmoechlisin702,20,2025-10-12,585-A9uh6Qc
4,Ugwt2EEv2dZFFy9kkql4AaABAg,Ayooo pak pur ttp semangat bangun indonesia 👍,@Emoxn7568,52,2025-10-12,585-A9uh6Qc
5,UgzKkoNBEX6sdhUJoLp4AaABAg,Ayo rakyat cerdas harus mengonyrol pemaik peng...,@jokoDS,1,2025-10-19,585-A9uh6Qc


## 9. Save Hasil Filter
Hasil filtering disimpan dalam dua file:  
- `dataset_filtered.csv` untuk data relevan  
- `dataset_not_used.csv` untuk data yang tidak relevan

In [34]:
OUTPUT_FILE_FILTER = "dataset_filtered.csv"
df_filter.to_csv(OUTPUT_FILE_FILTER, index=False, encoding='utf-8')
print(f"Berhasil menyimpan df_filter ke: {OUTPUT_FILE_FILTER}")

Berhasil menyimpan df_filter ke: dataset_filtered.csv


In [35]:
OUTPUT_FILE_FILTER = "dataset_not_used.csv"
df_not_used.to_csv(OUTPUT_FILE_FILTER, index=False, encoding='utf-8')
print(f"Berhasil menyimpan df_filter ke: {OUTPUT_FILE_FILTER}")

Berhasil menyimpan df_filter ke: dataset_not_used.csv


## 10. Kesimpulan
Tahap scraping dan filtering sudah berhasil dilakukan.  
Dataset `dataset_filtered.csv` siap digunakan untuk tahap **preprocessing teks**.